In [4]:
!pip install pandas numpy matplotlib scikit-learn statsmodels prophet tensorflow --quiet


In [6]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Optional: Confirm GPU is detected
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
# Load the datasets (Make sure you uploaded them in the Files panel on the left)

df_train = pd.read_csv("train_aWnotuB.csv")
df_test = pd.read_csv("datasets_8494_11879_test_BdBKkAj.csv")

print("Train Data:")
display(df_train.head())

print("\nTest Data:")
display(df_test.head())


Train Data:


,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041



Test Data:


,DateTime,Junction,ID
0,2017-07-01 00:00:00,1,20170701001
1,2017-07-01 01:00:00,1,20170701011
2,2017-07-01 02:00:00,1,20170701021
3,2017-07-01 03:00:00,1,20170701031
4,2017-07-01 04:00:00,1,20170701041


In [9]:
# Convert DateTime column to datetime type
df_train['DateTime'] = pd.to_datetime(df_train['DateTime'])
df_test['DateTime'] = pd.to_datetime(df_test['DateTime'])

# Sort the data correctly (very important for forecasting)
df_train = df_train.sort_values(['Junction', 'DateTime']).reset_index(drop=True)
df_test = df_test.sort_values(['Junction', 'DateTime']).reset_index(drop=True)

print("Preprocessing Done ✅")


Preprocessing Done ✅


In [10]:
# This function converts a continuous time-series into sequence windows
# Example: If look_back = 24 → use last 24 hours to predict the next hour
def create_sequences(series, look_back=24):
    X, y = [], []
    for i in range(len(series) - look_back):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back])
    return np.array(X), np.array(y)


# This function builds the LSTM model structure
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=False, input_shape=input_shape))
    model.add(Dropout(0.2))   # helps prevent overfitting
    model.add(Dense(1))       # output layer → predicts 1 future value
    model.compile(optimizer='adam', loss='mse')
    return model


In [11]:
# We will train one LSTM model per Junction (1,2,3,4)

look_back = 24       # use last 24 hours to predict next hour
epochs = 12          # you can increase later to improve accuracy
batch_size = 64
val_hours = 24 * 7   # last 7 days for validation

junctions = sorted(df_train['Junction'].unique().tolist())
lstm_models = {}     # to store model + scaler per junction
metrics = {}         # to store evaluation results

for j in junctions:
    print(f"\n🔹 Training LSTM for Junction {j} ...")

    # Select only one junction's data
    data_j = df_train[df_train['Junction'] == j]['Vehicles'].values.astype('float32').reshape(-1, 1)

    # Scale values (LSTM learns better when data is 0–1)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(data_j).flatten()

    # Split into training and validation
    train_end = len(scaled) - val_hours
    train_data = scaled[:train_end]
    val_data = scaled[train_end - look_back:]

    # Create sequences
    X_train, y_train = create_sequences(train_data, look_back)
    X_val, y_val = create_sequences(val_data, look_back)

    # Reshape → (samples, time steps, features)
    X_train = X_train.reshape(X_train.shape[0], look_back, 1)
    X_val = X_val.reshape(X_val.shape[0], look_back, 1)

    # Build model
    model = build_lstm_model((look_back, 1))
    es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[es]
    )

    # Evaluate model performance
    val_pred_scaled = model.predict(X_val).flatten()
    val_true = scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    val_pred = scaler.inverse_transform(val_pred_scaled.reshape(-1,1)).flatten()

    mae = mean_absolute_error(val_true, val_pred)
    rmse = math.sqrt(mean_squared_error(val_true, val_pred))

    metrics[j] = {"MAE": mae, "RMSE": rmse}
    lstm_models[j] = (model, scaler, scaled)

metrics



🔹 Training LSTM for Junction 1 ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

🔹 Training LSTM for Junction 2 ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

🔹 Training LSTM for Junction 3 ...


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



🔹 Training LSTM for Junction 4 ...


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


{1: {'MAE': 4.829691410064697, 'RMSE': 6.60251934633667},
 2: {'MAE': 2.753866195678711, 'RMSE': 3.5032115915368465},
 3: {'MAE': 2.713689088821411, 'RMSE': 3.7214813459841904},
 4: {'MAE': 2.2219862937927246, 'RMSE': 3.453493262629429}}

In [13]:
submission_rows = []

for j in junctions:
    print(f"\n🔻 Forecasting for Junction {j} ...")

    model, scaler, scaled_series = lstm_models[j]

    # Last 24 hours from the training data
    last_window = scaled_series[-look_back:].reshape(1, look_back, 1)

    # Number of predictions required for this junction
    test_j = df_test[df_test['Junction'] == j]
    steps = len(test_j)

    preds_scaled = []

    for _ in range(steps):
        pred = model.predict(last_window, verbose=0)[0, 0]
        preds_scaled.append(pred)

        # slide the window
        last_window = np.append(last_window[:, 1:, :], [[[pred]]], axis=1)

    # Convert predictions back to real vehicle counts
    preds = scaler.inverse_transform(np.array(preds_scaled).reshape(-1, 1)).flatten()

    # Ensure no negative predictions and round to nearest integer
    preds = np.clip(preds, 0, None)
    preds = np.round(preds).astype(int)

    # Store results with correct ID mapping
    submission_rows.append(pd.DataFrame({
        "ID": test_j["ID"].values,
        "Vehicles": preds
    }))

# Combine all junction predictions
submission = pd.concat(submission_rows).sort_values("ID").reset_index(drop=True)

submission.head()



🔻 Forecasting for Junction 1 ...

🔻 Forecasting for Junction 2 ...

🔻 Forecasting for Junction 3 ...

🔻 Forecasting for Junction 4 ...


,ID,Vehicles
0,20170701001,70
1,20170701002,26
2,20170701003,31
3,20170701004,13
4,20170701011,61


In [14]:
submission.to_csv("submission_lstm.csv", index=False)
print("✅ submission_lstm.csv file created successfully!")


✅ submission_lstm.csv file created successfully!
